# ทดสอบการค้นคืนข่าวที่น่าชื่อถือบน cofact ด้วย cosine similarity + TF-IDF (ใช้เฉพาะหัวข้อข่าวที่น่าเชื่อถือ(header จาก cofact_refer.json) 10/12/2021)

In [1]:
import numpy as np
import nltk
import re
import pandas as pd 
import json
import os
import pickle

from pythainlp import word_tokenize
from pythainlp.corpus import thai_stopwords
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
from stop_words import get_stop_words
from string import punctuation

from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import spatial

nltk.download('words')
th_stop = thai_stopwords()
en_stop = get_stop_words('en')
p_stemmer = PorterStemmer()

def split_word(text):
    tokens = word_tokenize(text,engine='newmm')
    
    # Remove stop words ภาษาไทย และอักขระพิเศษ
    tokens = [i for i in tokens if (not i in th_stop) & (not i in en_stop) & (not i in punctuation) & (not i in ["'",'"','“','”','‘','’','\n',"None", ' '])] 
    
    # ลบตัวเลข
    tokens = [i for i in tokens if not i.isnumeric()]
    
    # ลบช่องว่าง
    tokens = [i for i in tokens if not ' ' in i]

    return tokens

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
def read_cofact_refer():
    # Opening JSON file
    root_path = os.getcwd()
    path = os.path.join(root_path, '../Scrapy/Scrapy_project/spiders//fetch file/cofact_refer.json')
    f = open(path, encoding="utf8")
    
    # returns JSON object as
    # a dictionary
    cofact_refer_text_data = json.load(f)

    # Closing file
    f.close()
    return cofact_refer_text_data

In [3]:
cofact_refer_text_data = read_cofact_refer()

In [4]:
len(cofact_refer_text_data)

2206

In [5]:
def get_cofact_refer_info(cofact_refer_text_data): #สำหรับดึงหัวข้อข่าวอ้างอิง (header) และลิงของข่าวอ้างอิง (link)
    refer_text_list = []

    for refer_text in range(len(cofact_refer_text_data)):
        tmp = []
        tmp.append(cofact_refer_text_data[refer_text]['header'])
        tmp.append(cofact_refer_text_data[refer_text]['link'])
        refer_text_list.append(tmp)

    return refer_text_list

In [6]:
cofact_refer_text_list = get_cofact_refer_info(cofact_refer_text_data)

In [7]:
cofact_refer_text_list

[['เบื้องลึก ราคาข้าวตกต่ำ ชาวนาไทย ตกเป็นเครื่องมือการเมือง ไร้อำนาจต่อรอง',
  'https://www.thairath.co.th/scoop/interview/2235826'],
 ['1 พ.ย.วันแรกเปิดประเทศ นักท่องเที่ยวแห่เข้าไทยกว่า 60 เที่ยวบิน ',
  'https://www.thairath.co.th/news/local/central/2233121'],
 ['ลงทะเบียนคนละครึ่ง 1.1 แสนสิทธิ์เต็มแล้ว มีสิทธิ์อยู่ รับเพิ่ม 1,500 บาท',
  'https://www.thairath.co.th/business/economics/2233130'],
 ['สธ.แนะฉีดวัคซีน mRNA กระตุ้นเข็ม 3 เต็มโดส ฉีดเชื้อตายไม่เหมาะกระตุ้นเชื้อตายซ้ำ',
  'https://www.tnnthailand.com/news/covid19/93991/'],
 ['รถบรรทุกหยุดวิ่ง 20% ประท้วงน้ำมันแพง  ',
  'https://www.pptvhd36.com/news/%E0%B9%80%E0%B8%A8%E0%B8%A3%E0%B8%A9%E0%B8%90%E0%B8%81%E0%B8%B4%E0%B8%88/159590'],
 ['ชาวเน็ตแห่แชร์ วิธีการป้องกันการ “ถูกดูดเงินจากบัญชี” มีอะไรบ้างเช็กเลยที่นี่',
  'https://www.tnnthailand.com/news/socialtalk/93975/'],
 ['ข่าวปลอม! นามบัตรเคลือบยา สามารถทำให้สลบได้ในทันทีหากสัมผัส',
  'https://mgronline.com/factcheck/detail/9630000083963'],
 ['ยอดไอเดีย! นวัตกรรม "เก้าอี้ต

In [8]:
len(cofact_refer_text_list)

2206

## ------------------------------------------------------------------------------
## สร้าง dataframe ของ cofact สำหรับอ้างถึงตอนแนบลิงค์ url แสดงผล

In [9]:
cofact_refer_text = []
cofact_refer_url = []
for i in range(len(cofact_refer_text_list)):
    cofact_refer_text.append(cofact_refer_text_list[i][0]) #list ของส่วนเนื้อหาเท่านั้น
    cofact_refer_url.append(cofact_refer_text_list[i][1]) #list ของ url เท่านั้น


In [10]:
#ทำ list ให้เป็น dataframe
cofact_original_text_and_headline_news_df = pd.DataFrame(list(zip(cofact_refer_text, cofact_refer_url)), columns=["หัวข้อข่าวอ้างอิงของ cofact", "URL ข่าวอ้างอิงของ cofact"])

cofact_original_text_and_headline_news_df

,หัวข้อข่าวอ้างอิงของ cofact,URL ข่าวอ้างอิงของ cofact
0,เบื้องลึก ราคาข้าวตกต่ำ ชาวนาไทย ตกเป็นเครื่อง...,https://www.thairath.co.th/scoop/interview/223...
1,1 พ.ย.วันแรกเปิดประเทศ นักท่องเที่ยวแห่เข้าไทย...,https://www.thairath.co.th/news/local/central/...
2,ลงทะเบียนคนละครึ่ง 1.1 แสนสิทธิ์เต็มแล้ว มีสิท...,https://www.thairath.co.th/business/economics/...
3,สธ.แนะฉีดวัคซีน mRNA กระตุ้นเข็ม 3 เต็มโดส ฉีด...,https://www.tnnthailand.com/news/covid19/93991/
4,รถบรรทุกหยุดวิ่ง 20% ประท้วงน้ำมันแพง,https://www.pptvhd36.com/news/%E0%B9%80%E0%B8%...
...,...,...
2201,ปวดอักเสบจากโรคหมอนรองกระดูกสันหลังทับเส้นประส...,https://sure.oryor.com/index.php/detail/media_...
2202,ดื่มกาแฟวันละสามแก้วทำให้หน้าอกเล็กลงไม่จริง,https://sure.oryor.com/index.php/detail/media_...
2203,คิ้วดกดำด้วยการทาปิโตรเลียม เจลลี่ ไม่จริง,https://sure.oryor.com/index.php/detail/media_...
2204,มะนาวเกลือแก้ไมเกรน มั่วมาก อย่าไปเชื่อ,https://sure.oryor.com/index.php/detail/media_...


In [11]:
cofact_headline_tokens_list = [split_word(txt) for txt in cofact_refer_text] #นำ list ของเนื้อหามาตัดคำ

In [12]:
cofact_tokens_list_j = [','.join(tkn) for tkn in cofact_headline_tokens_list]

----------------------------------------------------------------------------------
# สร้าง vocabulary set

In [13]:
print(cofact_tokens_list_j[0].split(",")[0:2])


['เบื้องลึก', 'ราคา']


In [14]:
## Create Vocabulary
tokens_list = []

for words in cofact_tokens_list_j:
    # print(words)
    temp_list = words.split(",")
    # print(temp_list)
    for i in temp_list:
        tokens_list.append(i)

vocabulary = set(tokens_list)
vocabulary = list(vocabulary)

In [15]:
len(vocabulary)

4421

--------------------------------------------------------------------------------------

In [16]:
tvec = TfidfVectorizer(analyzer=lambda x:x.split(','),)
original_c_feat = tvec.fit_transform(cofact_tokens_list_j)

### Save Trained TFID Model

In [17]:
### Save model
root_path = os.getcwd()
path = os.path.join(root_path, 'cofact-tfid.pkl')
print(path)
with open(path,'wb') as handle:
    pickle.dump(original_c_feat, handle)

c:\Users\Administrator\Documents\SeniorProject\Senior-project\Pre-process\cofact-tfid.pkl


In [18]:
### load model
root_path = os.getcwd()
path = os.path.join(root_path, 'cofact-tfid.pkl')
print(path)
t = pickle.load(open(path,'rb'))

c:\Users\Administrator\Documents\SeniorProject\Senior-project\Pre-process\cofact-tfid.pkl


### Save Vocabulary

In [19]:
### Save Vacabulary
root_path = os.getcwd()
path = os.path.join(root_path, 'vocabulary_cofact.txt')
print(path)
with open(path, "w", encoding="utf-8") as file:
    file.write(str(vocabulary))

c:\Users\Administrator\Documents\SeniorProject\Senior-project\Pre-process\vocabulary_cofact.txt


In [20]:
### load Vacabulary
root_path = os.getcwd()
path = os.path.join(root_path, 'vocabulary_cofact.txt')
print(path)
with open(path, "r", encoding="utf-8") as file:
    data2 = eval(file.readline())

c:\Users\Administrator\Documents\SeniorProject\Senior-project\Pre-process\vocabulary_cofact.txt


In [21]:
data2

['',
 'หารายได้',
 'ป.ป.ส.',
 'ปลอด',
 'การคลัง',
 'กระดูกพรุน',
 'ตากแดด',
 'มาลำ',
 'ฝนเทียม',
 'วจะ',
 'ระงับ',
 '3.61',
 '"ฝน',
 'ถอดความ',
 'Alternative',
 'สธ',
 'อาละวาด',
 'ยอมให้',
 'ร้านอาหาร',
 'ดูแล',
 'โต๊ะ',
 '“แซนด์',
 'พริกไทย',
 'Mia',
 'ท้อใจ',
 'ปั๋ง',
 'ถก',
 'นรีแพทย์',
 'ปูพรม',
 'อีเมล์',
 'บอนด์',
 'โปร',
 'ผสม',
 'เป๋า',
 'ผู้เข้าร่วม',
 'เอาผิด',
 'น้ำปลา',
 'ลือ',
 'ลเล็ต',
 'ประสิทธิ์',
 'Update',
 'กรมราชทัณฑ์',
 'มหาชัย',
 'ข้าวหมาก',
 '1.1',
 'หมุน',
 'ผิวเหลือง',
 'อิเล็กทรอนิกส์',
 'อาหารเสริม',
 'ภิ',
 'ฟาร์ม',
 'ไผ่',
 'สบู่เหลว',
 'ที่วัด',
 'สรุป',
 'ป้อม',
 'ซู',
 'ลูกปืน',
 'ป้อง',
 'โรคหวัด',
 'แอบอ้าง',
 'เน่า',
 'ทบ.',
 '\u200bปม',
 'ขจัดคราบ',
 'สาว',
 'ว่าที่',
 'สังเคราะห์',
 'วีค',
 '“รถ',
 'ถอดใจ',
 'สกัด',
 'ไบค์',
 'กระดูกคอ',
 'หมุด',
 'แวค"',
 'ประกันชีวิต',
 'E',
 'บอด',
 'มากขึ้น',
 'คุณแม่',
 'เซิร์ฟ',
 'รู้ตัว',
 'คลอด',
 'แบบสอบถาม',
 'รว์"',
 'ลมพิษ',
 'โควิด',
 'จอง',
 'เบกกิ้ง',
 'ไขมัน',
 'ภาวะ',
 'ครอบจักรวาล',
 'ทะลาย',
 'ช่

In [22]:
len(data2)

4421

## Create vector for Query/search keywords

In [23]:
## Create vector for Query/search keywords
def gen_vector_T(tokens):

    Q = np.zeros((len(vocabulary)))
    
    x = tvec.transform(tokens)
    
    x = x.A[0]
    return x

### Calculate Cosine similarity of trained Tfidf to input query


In [24]:
query_vector = []
def cosine_similarity_T(k, query):
    tokens = split_word(str(query))
    q_df = pd.DataFrame(columns=['q_clean'])
    # print(tokens)
    q_df.loc[0,'q_clean'] =str(tokens)

    q_df=q_df.replace(to_replace ="\[.", value = '', regex = True)
    q_df=q_df.replace(to_replace ="'", value = '', regex = True)
    q_df=q_df.replace(to_replace =" ", value = '', regex = True)
    q_df=q_df.replace(to_replace ='\]', value = '', regex = True)
    
    d_cosines = []
    
    query_vector = gen_vector_T(q_df['q_clean'])
    
    for d in original_c_feat.A:
        d_cosines.append(1 - spatial.distance.cosine(query_vector, d))
    
    out = np.array(d_cosines).argsort()[-k:][::-1]
    d_cosines.sort()

    a = pd.DataFrame()
    for i in out:
        a.loc[i,'index'] = str(i)
        a.loc[i,'Subject'] = cofact_original_text_and_headline_news_df['หัวข้อข่าวอ้างอิงของ cofact'][i]

    d_cosines.sort(reverse=True)
    
    for j in range(k):
        a.loc[out[j],'Score'] = d_cosines[j]
        
    return a

In [29]:
%time cofact_result = cosine_similarity_T(10,'จริงไหมตอนนี้ชาวนาขายข้าวต่อกิโลได้เท่ากับค่าบะหมี่กึ่งสำเร็จรูปซองเดียว')

Wall time: 1.83 s


In [30]:
cofact_result

,index,Subject,Score
680,680,ประกาศจุฬาราชมนตรี แจ้งเตือน ผลิตภัณฑ์อาหารกึ่...,0.232745
0,0,เบื้องลึก ราคาข้าวตกต่ำ ชาวนาไทย ตกเป็นเครื่อง...,0.209275
424,424,รู้จัก “โรคไข้หูดับ” กินเนื้อหมู กึ่งสุกกึ่งดิ...,0.196548
427,427,รู้จัก “โรคไข้หูดับ” กินเนื้อหมู กึ่งสุกกึ่งดิ...,0.196548
1766,1766,ก.เกษตรฯจับมือไปรษณีย์ไทย ส่ง EMSผลไม้แค่ 8 บา...,0.159673
98,98,"ถึงคิว ""ชาวนา"" เตรียมรับ ""เงินเยียวยา"" ไร่ละ 1...",0.153760
695,695,เชื้อโควิด-19 สามารถแพร่กระจายผ่านทางอากาศได้ไหม,0.153407
362,362,จับพัสดุปนเปื้อน COVID-19 จะติดโรคไหม?,0.142893
389,389,จับพัสดุปนเปื้อน COVID-19 จะติดโรคไหม?,0.142893
660,660,ติดโควิด รักษาครบ 14 วัน ต้องตรวจหาเชื้อก่อนกล...,0.142063


In [27]:
cofact_result_with_url = pd.DataFrame()
for i in range(len(cofact_result)):
    cofact_result_with_url.loc[i,'index'] = cofact_result.iloc[i]["index"]
    cofact_result_with_url.loc[i,'text'] = cofact_result.iloc[i]["Subject"]
    cofact_result_with_url.loc[i,'url'] = cofact_original_text_and_headline_news_df["URL ข่าวอ้างอิงของ cofact"][int(cofact_result.iloc[i]["index"])]

In [28]:
cofact_result_with_url

,index,text,url
0,15,"เพจดังเตือนระวัง ""กาแฟผสมยาอี"" ฮิตใน Tiktok ป...",https://news.thaipbs.or.th/content/309283
1,2173,กาแฟ ต้านโควิดได้ จริงหรือไม่,https://sure.oryor.com/index.php/detail/media_...
2,2022,อย.เตือน! ระวังกาแฟผสมยาปลุกเซ็กซ์ ส่งผลเสียต่...,https://www.matichon.co.th/local/quality-life/...
3,1316,ข่าวปลอม อย่าแชร์! กาแฟผสมมะนาว น้ำมันมะพร้าว ...,https://www.antifakenewscenter.com/%e0%b8%82%e...
4,1520,WHO ชี้วัยรุ่นหนุ่ม-สาว ติดเชื้อโควิดพุ่ง 3 เท่า,https://www.tnnthailand.com/news/covid19/50198/
5,2167,แชร์ผิดผิด กาแฟใส่เนย,https://oryor.com/%E0%B8%AD%E0%B8%A2/detail/me...
6,1475,ข่าวปลอม อย่าแชร์! ดื่มกาแฟดำผสมมะนาว เเก้ปวดไ...,https://www.antifakenewscenter.com/%e0%b8%82%e...
7,833,แพทย์เตือนวัยรุ่นไทย ใช้กลูต้าฯเพื่อผิวขาว เสี...,https://www.thairath.co.th/content/223039
8,1645,ดื่มชาปลอดภัยกว่ากาแฟจริงหรือ,https://sure.oryor.com/index.php/detail/media_...
9,793,กาแฟกับภาวะกระดูกพรุน,https://oryor.com/%E0%B8%AD%E0%B8%A2/detail/me...


---------------------------------------